# **Amazon Nova 2 Lite Model**

In [1]:
import os
import boto3
import json
import base64
from PIL import Image
from IPython.display import Markdown, display
bedrockruntime = boto3.client('bedrock-runtime', region_name='us-west-2')
novalite = 'us.amazon.nova-2-lite-v1:0'

# **Define Nova Lite prompt function (.pdf file)**

In [2]:
def pdfprompt(pdfpath, prompt, maxTokens, temperature):
    # Read and base64 encode the image
    with open(pdfpath, "rb") as file:
        docs_bytes = file.read()
        base64_string = base64.b64encode(docs_bytes).decode("utf-8")

    extension = os.path.splitext(pdfpath)[1][1:]
    filename = os.path.splitext(pdfpath)[0]

    # Create the message structure
    message = [
        {
            "role": "user",
            "content": [
                {
                    "document": {
                        "format": extension,
                        "name": filename,
                        "source": {
                            "bytes": base64_string
                        }
                    }
                },
                {
                    "text": prompt
                }
            ],
        }
    ]

    inf_params = {"maxTokens": maxTokens, "temperature": temperature}

    request = {
        "messages": message,
        "inferenceConfig": inf_params
    }

    response = bedrockruntime.invoke_model(modelId=novalite, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    return model_response

# **Prompt Engineering - Recommendation / Decision Making**

In [3]:
pdfpath = "Always Winner CV.pdf"

minimal_requirements = '''
Bachelor's or Master's degree in Computer Science, Engineering, Mathematics, or a related field
Min. 1 year of work experience in developing and implementing AI solutions
Proficient in frameworks such as TensorFlow, PyTorch, and programming languages like Python, R, Java, or C++
Skilled in RAG (Retrieval-Augmented Generation) techniques and LLMs (Large Language Models)
Basic knowledge of application development and deployment processes
In-depth understanding of machine learning algorithms, deep learning, and other AI techniques
Experience with data preprocessing and data analysis techniques
Ability to evaluate and improve AI model performance
'''.strip()

preferred_requirements = '''
Experienced with cloud services such as AWS, Google Cloud, or Azure
'''.strip()

In [4]:
prompt = "Do you recommend this candidate for next step based on {minimum_requirements} and {preferred_requirements}? Answer YES or NO and simple recommendation/decision making."
maxTokens = 1000
temperature = 0.1
pdfresult = pdfprompt(pdfpath, prompt, maxTokens, temperature)
print(pdfresult)

{'output': {'message': {'content': [{'text': "**YES**\n\n### Recommendation/Decision Making:\nBased on the provided resume, the candidate meets or exceeds both the **minimum requirements** and **preferred requirements** for an AI Engineer role. Here's a brief rationale:\n\n1. **Education**: The candidate holds a **Bachelor of Artificial Intelligence** from a recognized institution, which aligns with typical minimum educational requirements for AI roles.\n   \n2. **Work Experience**: The candidate has **relevant work experience** in AI engineering, including roles as both a **Junior AI Engineer** and an **AI Engineer**, demonstrating progression and hands-on experience in the field.\n\n3. **Technical Skills**:\n   - **Programming Languages**: Proficiency in **Python** (with key libraries like TensorFlow, PyTorch, etc.) and additional languages like **Java** and **C++** are strong indicators of technical capability.\n   - **Generative AI**: Experience with **GANs, VAEs, Diffusion Models,

In [5]:
answer = pdfresult["output"]["message"]["content"][0]["text"]
display(Markdown(answer))

**YES**

### Recommendation/Decision Making:
Based on the provided resume, the candidate meets or exceeds both the **minimum requirements** and **preferred requirements** for an AI Engineer role. Here's a brief rationale:

1. **Education**: The candidate holds a **Bachelor of Artificial Intelligence** from a recognized institution, which aligns with typical minimum educational requirements for AI roles.
   
2. **Work Experience**: The candidate has **relevant work experience** in AI engineering, including roles as both a **Junior AI Engineer** and an **AI Engineer**, demonstrating progression and hands-on experience in the field.

3. **Technical Skills**:
   - **Programming Languages**: Proficiency in **Python** (with key libraries like TensorFlow, PyTorch, etc.) and additional languages like **Java** and **C++** are strong indicators of technical capability.
   - **Generative AI**: Experience with **GANs, VAEs, Diffusion Models, and LLMs** shows expertise in cutting-edge AI techniques.
   - **Machine Learning**: Knowledge of **supervised/unsupervised learning, transfer learning, model optimization, and hyperparameter tuning** is essential for AI roles.
   - **Cloud Platforms**: Familiarity with **AWS, Azure, and Google Cloud** is a significant plus, as cloud expertise is highly valued in modern AI roles.
   - **Tools & Libraries**: Experience with **Docker, Kubernetes, Git, MLflow, etc.** indicates strong DevOps and MLOps skills.
   - **Databases**: Knowledge of **SQL and NoSQL** databases is important for data handling and management.
   - **Other Skills**: Exposure to **MLOps, Data Engineering, API Development, and Agile Methodologies** further strengthens the candidate's profile.

Given the comprehensive skill set, relevant experience, and alignment with both minimum and preferred requirements, this candidate is highly recommended for the next step in the hiring process.

# **Define Nova Lite prompt function (.png or .jpg file)**

In [6]:
def imageprompt(imagepath, prompt, maxTokens, temperature):
    # Read and base64 encode the image
    with open(imagepath, "rb") as file:
        image_bytes = file.read()
        base64_string = base64.b64encode(image_bytes).decode("utf-8")

    extension = os.path.splitext(imagepath)[1][1:]

    # Create the message structure
    message = [
        {
            "role": "user",
            "content": [
                {
                    "image": {
                        "format": extension,
                        "source": {
                            "bytes": base64_string
                        },
                    }
                },
                {
                    "text": prompt
                }
            ],
        }
    ]

    inf_params = {"maxTokens": maxTokens, "temperature": temperature}

    request = {
        "messages": message,
        "inferenceConfig": inf_params
    }

    response = bedrockruntime.invoke_model(modelId=novalite, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    return model_response

# **Prompt Engineering - Structured Output**

In [7]:
imagepath = "receipts.jpg"
prompt = '''
Extract this receipt and write result of extraction with format:
- Store Name :
- Date :
- Total :
'''.strip()
maxTokens = 300
temperature = 0.1
imageresult = imageprompt(imagepath, prompt, maxTokens, temperature)
print(imageresult)

{'output': {'message': {'content': [{'text': '- Store Name : Pasta & Co.\n- Date : 08-12-2023\n- Total : $29.80'}], 'role': 'assistant'}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 1679, 'outputTokens': 29, 'totalTokens': 1708, 'cacheReadInputTokenCount': 0, 'cacheWriteInputTokenCount': 0}}


In [8]:
answer = imageresult["output"]["message"]["content"][0]["text"]
display(Markdown(answer))

- Store Name : Pasta & Co.
- Date : 08-12-2023
- Total : $29.80

# **Prompt Engineering - Content Generation**

In [9]:
def contentprompt(prompt, maxTokens, temperature):
    message = [
        {
            "role": "user",
            "content": [
                {
                    "text": prompt
                }
            ]
        }
    ]

    inf_params = {"maxTokens": maxTokens, "temperature": temperature}

    request = {
        "messages": message,
        "inferenceConfig": inf_params
    }

    response = bedrockruntime.invoke_model(modelId=novalite, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    return model_response

In [10]:
prompt = '''
Create a marketing campaign to attract customers based this data:
- Topic : Cheap landed house in strategic location.
- Target and age : Milenial and Gen Z, between 19 - 40.
- Tone : Detailed informative, invite customers to buy.
'''.strip()
maxTokens = 2000
temperature = 0.1
result = contentprompt(prompt, maxTokens, temperature)
print(result)

{'output': {'message': {'content': [{'text': '### **Marketing Campaign: “Your Dream Home, Without the Dream Price”**  \n**Target Audience:** Millennials & Gen Z (Ages 19–40)  \n**Topic:** Affordable Landed Houses in Strategic Locations  \n**Tone:** Detailed, Informative, and Inviting — *Helping Young Adults Find Smart, Budget-Friendly Homes in Prime Areas*\n\n---\n\n## 🎯 **Campaign Overview**\n\nWe’re launching a multi-channel marketing campaign designed to attract **young professionals, first-time homebuyers, and growing families** who want **quality, affordable landed houses** in **strategic, well-connected locations**. The campaign will blend **detailed information**, **real-life testimonials**, and **interactive experiences** to build trust and drive action.\n\n---\n\n## 🧭 **Campaign Goals**\n\n1. **Raise Awareness** about affordable landed housing options in prime areas.\n2. **Generate Leads** through inquiries, brochure downloads, and virtual tours.\n3. **Drive Sales** by convert

In [11]:
answer = result["output"]["message"]["content"][0]["text"]
display(Markdown(answer))

### **Marketing Campaign: “Your Dream Home, Without the Dream Price”**  
**Target Audience:** Millennials & Gen Z (Ages 19–40)  
**Topic:** Affordable Landed Houses in Strategic Locations  
**Tone:** Detailed, Informative, and Inviting — *Helping Young Adults Find Smart, Budget-Friendly Homes in Prime Areas*

---

## 🎯 **Campaign Overview**

We’re launching a multi-channel marketing campaign designed to attract **young professionals, first-time homebuyers, and growing families** who want **quality, affordable landed houses** in **strategic, well-connected locations**. The campaign will blend **detailed information**, **real-life testimonials**, and **interactive experiences** to build trust and drive action.

---

## 🧭 **Campaign Goals**

1. **Raise Awareness** about affordable landed housing options in prime areas.
2. **Generate Leads** through inquiries, brochure downloads, and virtual tours.
3. **Drive Sales** by converting interested leads into booked viewings and signed contracts.
4. **Build Brand Trust** by positioning the developer as transparent, customer-focused, and community-oriented.

---

## 📊 **Target Audience Insights**

| Demographic | Millennials (25–40) | Gen Z (19–24) |
|------------|----------------------|----------------|
| **Motivations** | Stability, space, investment, family | Affordability, location, social status, flexibility |
| **Pain Points** | High property prices, limited budget, long commutes | Limited savings, student debt, desire for independence |
| **Media Habits** | Instagram, Facebook, LinkedIn, YouTube, WhatsApp | TikTok, Instagram Reels, YouTube Shorts, Snapchat |
| **Decision Triggers** | Clear ROI, flexible payment plans, prime location | Aesthetic appeal, social proof, instant access, peer reviews |

---

## 📣 **Campaign Tagline & Messaging**

### **Main Tagline:**
> **“Your Dream Home, Without the Dream Price.”**

### **Supporting Messages:**
- **“Live Smart. Live Central. Live Affordable.”**
- **“First Home? We’ve Got You Covered.”**
- **“Strategic Location, Budget-Friendly Price — It’s Possible.”**
- **“From Apartment to Landed — Without Breaking the Bank.”**
- **“Invest in Your Future. Start with Your Home.”**

---

## 📅 **Campaign Timeline (12-Week Plan)**

| **Week** | **Activity** | **Channels** |
|----------|--------------|---------------|
| 1–2 | **Teaser Phase** – Mysterious countdown posts, “Something Big is Coming” | Instagram, TikTok, Facebook, Email |
| 3 | **Launch Week** – Campaign launch video, landing page go-live | All social media, website, YouTube |
| 4–6 | **Awareness & Education** – Detailed content series on affordability, location benefits | Blog, Instagram carousel, YouTube, webinars |
| 7–8 | **Engagement & Interaction** – Virtual tours, live Q&A, user-generated content | Instagram Live, TikTok, Facebook Live |
| 9–10 | **Conversion Focus** – Limited-time offers, early-bird discounts, referral program | Email, SMS, WhatsApp, retargeting ads |
| 11–12 | **Closing & Retention** – Success stories, post-purchase support, community building | Email newsletters, follow-up calls, loyalty program |

---

## 📱 **Multi-Channel Marketing Strategy**

### 1. **Social Media Marketing**

#### **Platforms:**
- **Instagram & Facebook** – Visual storytelling, reels, carousel posts, stories, polls
- **TikTok** – Short, catchy videos showing house tours, price breakdowns, and “day in the life” of residents
- **YouTube** – Longer-form videos: virtual tours, developer interviews, buyer testimonials
- **Snapchat** – Geo-filters for nearby areas, AR house previews

#### **Content Types:**
- **“Why This House?” Series** – 60-second videos explaining features, location perks, and price justification
- **“Budget Breakdown” Infographics** – How much you *really* need to afford a landed house
- **“Meet the Neighbors”** – Real residents sharing their experiences
- **“Myth vs Fact” Posts** – Debunking common misconceptions about affordable housing
- **Interactive Polls & Quizzes** – “Which Home Fits Your Lifestyle?”

---

### 2. **Content Marketing & SEO**

#### **Website & Blog**
- **Dedicated Campaign Landing Page**  
  - Clear CTA: “Book a Free Virtual Tour”
  - Interactive map showing nearby schools, transport, malls
  - Downloadable PDF: “The Young Buyer’s Guide to Affordable Landed Homes”

#### **SEO Strategy**
- Target keywords:  
  - “cheap landed house near me”  
  - “affordable house for first-time buyers”  
  - “strategic location landed property”  
  - “budget landed house Malaysia/Singapore/Indonesia” *(adjust by country)*

#### **Blog Content Ideas**
- “5 Mistakes Young Buyers Make (And How to Avoid Them)”
- “How to Buy Your First Landed House on a Tight Budget”
- “Why Location Matters More Than You Think”
- “The Hidden Costs of Buying a Home – And How We Help You Manage Them”

---

### 3. **Email & SMS Marketing**

#### **Segments:**
- **First-time buyers**
- **Young professionals**
- **Existing followers**
- **Referral program members**

#### **Email Series:**
1. **Welcome Series** – Introduction to the campaign + downloadable guide
2. **Location Deep Dive** – Why this area is perfect for you
3. **Price Breakdown** – How you can own a landed house for X amount
4. **Success Stories** – Real buyers just like you
5. **Limited-Time Offer** – Early-bird discount or flexible payment plan

#### **SMS Campaigns**
- “Hi [Name]! Your dream home is now within reach. Book a FREE virtual tour today: [link]”

---

### 4. **Paid Advertising**

#### **Platforms:**
- **Meta (Facebook & Instagram Ads)** – Targeted by age, location, interests (real estate, home improvement, first-time buyers)
- **Google Ads** – Search & display campaigns targeting high-intent keywords
- **TikTok Ads** – In-feed ads, branded hashtag challenge: #MyFirstLanded
- **YouTube Ads** – Skippable video ads before home-related content

#### **Ad Creatives:**
- **Video Ads:** 15–30s clips showing quick house tours, price tags, location perks
- **Carousel Ads:** “Before & After” of developed areas, “5 Reasons to Buy Now”
- **Retargeting Ads:** For users who visited the landing page but didn’t convert

---

### 5. **Partnerships & Influencer Collaborations**

#### **Micro-Influencers (5k–100k followers)**
- Real estate influencers
- Lifestyle influencers (young families, budget travelers, urban professionals)
- Finance YouTubers / bloggers who talk about first-time homebuying

#### **Collaboration Ideas:**
- “Day in the Life” vlog of living in one of our properties
- “How I Bought My First Landed House at 28” – sponsored post
- Giveaway contest: “Win a Free Interior Design Consultation”

#### **Community Partnerships**
- Local universities & colleges – Career fairs, housing fairs
- Co-working spaces – Posters, QR codes for virtual tours
- Startup incubators – Offer discounted packages for young entrepreneurs

---

### 6. **Events & Experiences**

#### **Virtual Open House Weekend**
- Live-streamed tours with Q&A
- Interactive 3D walkthroughs
- Chat with sales advisors in real-time

#### **Pop-Up Booth at Shopping Malls**
- In strategic malls near target neighborhoods
- Free goodie bags, QR codes for instant quote
- Spin-to-win discount wheel

#### **First-Time Homebuyer Workshop**
- Free webinar or in-person seminar
- Topics: “How to Finance Your First Home,” “What to Look for in a Location,” “Understanding Legal Terms”

---

## 💰 **Incentives & Offers**

### **For First-Time Buyers:**
- **Zero Down Payment Option** *(subject to credit approval)*
- **Flexible Payment Plan** – Up to 10 years
- **Free Interior Design Consultation**
- **Waived Legal Fees** – Save up to RM 5,000

### **Referral Program:**
- Refer a friend who buys → **RM 2,000 voucher** for home decor or furniture

### **Limited-Time Launch Offer:**
- **First 10 buyers** get **10% discount** or **free parking space**

---

## 📈 **Success Metrics & KPIs**

| **Metric** | **Target** |
|------------|------------|
| Website Traffic | 50,000 unique visitors |
| Social Media Reach | 1M impressions |
| Email List Growth | 10,000 new subscribers |
| Lead Generation | 2,000 qualified leads |
| Virtual Tour Bookings | 1,000 bookings |
| Conversion Rate | 5% from lead to sale |
| Social Engagement | 10% engagement rate |
| Influencer Reach | 500K+ impressions |
| Event Attendance | 500+ attendees (virtual & physical) |

---

## 🧾 **Budget Allocation (Sample)**

| **Channel** | **Budget Allocation** |
|-------------|------------------------|
| Social Media Ads | 35% |
| Influencer Marketing | 20% |
| Content & SEO | 15% |
| Email & SMS | 10% |
| Events & Pop-ups | 10

# **Prompt Engineering - QnA (Question and Answering)**

In [13]:
imagepath = "bedrock.png"
prompt = '''
1. What open source protocol products?
2. What open source framework products?
3. What features of Bedrock Agentcore?
Answering with product name only.
'''
maxTokens = 300
temperature = 0.1
imageresult = imageprompt(imagepath, prompt, maxTokens, temperature)
print(imageresult)

{'output': {'message': {'content': [{'text': '1. MCP, A2A\n2. Strands Agents, CrewAI, LangGraph, OpenAI Agents SDK, Claude SDK\n3. Runtime, Browser, Memory, Observability, Identity, Policy, Gateway, Evaluations, Code Interpreter'}], 'role': 'assistant'}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 2636, 'outputTokens': 52, 'totalTokens': 2688, 'cacheReadInputTokenCount': 0, 'cacheWriteInputTokenCount': 0}}


In [15]:
answer = imageresult["output"]["message"]["content"][0]["text"]
display(Markdown(answer))

1. MCP, A2A
2. Strands Agents, CrewAI, LangGraph, OpenAI Agents SDK, Claude SDK
3. Runtime, Browser, Memory, Observability, Identity, Policy, Gateway, Evaluations, Code Interpreter

# **Zero-Shot Prompt Technique**

In [16]:
def promptechnique(prompt, maxTokens, temperature):
    message = [
        {
            "role": "user",
            "content": [
                {
                    "text": prompt
                }
            ]
        }
    ]

    inf_params = {"maxTokens": maxTokens, "temperature": temperature}

    request = {
        "messages": message,
        "inferenceConfig": inf_params
    }

    response = bedrockruntime.invoke_model(modelId=novalite, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    return model_response

In [18]:
prompt = """
Classify the sentiment (Positive, Negative, Neutral) of the following customer feedback without explanation.
Your new feature is best in your product. This is very recommended.
"""
maxTokens = 50
temperature = 0.1
result = promptechnique(prompt, maxTokens, temperature)
answer = result["output"]["message"]["content"][0]["text"]
display(Markdown(answer))

**Positive**

The feedback expresses clear approval and enthusiasm for the new feature, using words like "best" and "very recommended," which indicate a positive sentiment.

# **One- or Few- Shot Prompt Technique**

In [21]:
prompt = """
Extract the product name from the text.
Text : I just bought the new Quantum Leap X1 smartphone, and it's amazing!
Product name : Quantum Leap X1

Text : We are happy to announce the release of the EverBloom Garden Sprinkler.
Product name :
"""
maxTokens = 50
temperature = 0.1
result = promptechnique(prompt, maxTokens, temperature)
answer = result["output"]["message"]["content"][0]["text"]
display(Markdown(answer))

### Product name : **EverBloom Garden Sprinkler**

---

**Explanation:**  
In the given text — *"We are happy to announce the release of the EverBloom Garden Sprinkler."* — the product being referenced is clearly named **EverBloom

# **Chain-of-Thought Prompt Technique**

In [22]:
prompt = """
You are must explain thinking detail step-by-step like reasoning process and show result of number.
A shop sell laptop for 5000 USD each. If Budi buys 3 laptops and pay 20000 USD, how much change will Budi receive?
"""
maxTokens = 2000
temperature = 0.1
result = promptechnique(prompt, maxTokens, temperature)
answer = result["output"]["message"]["content"][0]["text"]
display(Markdown(answer))

### Step-by-Step Reasoning:

We are given the following information:

- **Price of each laptop** = 5000 USD  
- **Budi buys** = 3 laptops  
- **Budi pays** = 20,000 USD  

We are asked to find **how much change Budi will receive**.

---

### Step 1: Calculate the **total cost** of 3 laptops

We multiply the price of one laptop by the number of laptops Budi buys:

$$
\text{Total cost} = \text{Price per laptop} \times \text{Number of laptops}
$$

$$
\text{Total cost} = 5000 \, \text{USD} \times 3 = 15000 \, \text{USD}
$$

---

### Step 2: Calculate the **change** Budi will receive

Change is calculated by subtracting the **total cost** from the **amount Budi paid**:

$$
\text{Change} = \text{Amount paid} - \text{Total cost}
$$

$$
\text{Change} = 20000 \, \text{USD} - 15000 \, \text{USD} = 5000 \, \text{USD}
$$

---

### ✅ Final Answer:

$$
\boxed{5000}
$$